# 1. Scrapy là gì

Scrapy là một framework được viết bằng Python, nó cấp sẵn 1 cấu trúc tương đối hoàn chỉnh để thực hiện việc crawl và extract data từ website một cách nhanh chóng và dễ dàng. Ví dụ như lấy toàn bộ hình ảnh trên 1 website; các bài viết trên các trang báo; thông tin dữ liệu các sản phẩm moblie, ô tô; các thông tin public trên facebook;...

Người dùng chỉ cần bổ sung thêm định nghĩa về dữ liệu cần lấy là xong, ví dụ như URL bắt đầu là gì, link chuyển qua trang mới, các thông tin cần lấy ở mỗi trang là gì...

# 2. Cài đặt

Scrapy chạy trên Python 2.7 và Python 3.5 trở lên trong CPython và PyPy 5.9 trở lên.

Nếu như bạn sử dụng Anaconda hoặc Miniconda, bạn có thể cài đặt gói từ conda-forge channel. Để cài đặt Scrapy sử dụng `conda`, chạy:

`conda install -c conda-forge scrapy`

Ngoài ra, nếu bạn đã quen với việc cài đặt các gói Python, bạn có thể cài đặt Scrapy từ PyPI với:

`pip install Scrapy`

# 3. Các khái niệm cơ bản

## 3.1: Command line tool

Scrapy được điều khiển thông qua command-line tool. Scrapy cung cấp một số lệnh, với nhiều mục đích thì sẽ sử dụng các nhóm lệnh khác nhau.

Đây là cấu trúc thư mục khi vừa tạo thông qua scrapy:

`scraper/
├── scraper                         # nơi chứa code của dự án
│   ├── __init__.py
│   ├── items.py                    # nơi định nghĩa các trường dữ liệu cần lưu vào db
│   ├── pipelines.py                # nơi xử lý các item trích xuất được và lưu vào db
│   ├── settings.py                 # cấu hình thêm các phần mở rộng (middlewares) và các thông số cấu hình khác
│   └── spiders                     # thư mục chứa các spider
│       └── __init__.py
└── scrapy.cfg                      # file cấu hình về deploy và settings của project`

Một số lệnh command-line:

<img src="commandline.PNG">

## 3.2: Spiders

Spiders là class được viết bởi người dùng, xác định một hoặc một nhóm trang web nhất định sẽ được quét, bao gồm cách thực hiện thu thập thông tin và cách trích xuất dữ liệu có cấu trúc từ các trang web đó.

Đối với Spiders, quá trình thu thập dữ liệu có chu trình như sau:

1. Bạn bắt đầu bằng cách tạo các yêu cầu bạn đầu để thu thập dữ liệu từ URL đầu tiên và chỉ định chức năng gọi lại được gọi với phản hồi từ các yêu cầu đó.

2. Trong hàm gọi lại, bạn phân tích cú pháp phản hồi và trả về các ký tự với dữ liệu được trích xuất.  Những yêu cầu đó cũng có thể chứa một yêu cầu gọi lại, sau đó cũng sẽ được Scrapy duyệt và phản hồi lại yêu cầu đó.

3. Cuối cùng, các mục được trả về từ spiders sẽ được duy trì trong CSDL hoặc đươc ghi vào một tệp bằng cách sử dụng Feed exports

### scrapy.Spider

Một số phương thức và thuộc tính chính:

<img src="method_scrapy.PNG">

Ví dụ:

In [1]:
import scrapy
class MySpider(scrapy.Spider):
    name = 'example'
    allowed_domains = ['example.com']
    start_urls = [
        'http://www.example.com/1.html',
        'http://www.example.com/2.html',
        'http://www.example.com/3.html',
    ]
    
    def parse(self, response):
        for h3 in response.xpath('//h3').getall():
            yield {"title": h3}
        for href in response.xpath('//a/@href').getall():
            yield scrapy.Request(response.urljoin(href), self.parse)

Sau đó sẽ chạy lệnh ở command line để thực hiện lấy các thông tin mà spider trích xuất.

`scrapy crawl example`

## 3.3: Selector

Khi quét các trang web, tác vụ phổ biến nhất cần thực hiện là trích xuất dữ liệu từ nguồn HTML. Scrapy có cơ chế riêng để trích xuất dữ liệu. Chúng chọn một số thành phần nhất định của tài liệu HTML được chỉ định bởi các biểu thức XPath hoặc CSS. XPath để chọn các nút trong tài liệu XML, cũng có thể được sử dụng cho các tài liệu HTML. CSS là ngôn ngữ để áp dụng các kiểu cho các tài liệu HTML.

Truy vấn các phản hồi bằng XPath và CSS ta sử dụng: `response.xpath()` và `response.css()`

Ví dụ:
<br/>
`>>> response.xpath('//span/text()').get()`
<br/>
'good'
<br/>
`>>>response.css('span::text').get()`
<br/>
'good'


### Một số phương thức

<img src="method_extract.PNG">

### CSS Selectors

In [15]:
from scrapy import Selector
sel = Selector(text="""
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>""")

<i><b>Chú ý:</b> Khi thực hiện crawl trên web thì phần text của response chính là mã nguồn đc scrapy tải xuống  hay nói cách khác response = sel. </i>

 <b>title::text</b>: Trích chọn text trong thẻ title

In [16]:
sel.css('title::text').extract_first()

'Example website'

<b>*::text</b> chọn tất cả text ở trong các thẻ

In [17]:
sel.css('#images *::text').getall()

['\n   ',
 'Name: My image 1 ',
 '\n   ',
 'Name: My image 2 ',
 '\n   ',
 'Name: My image 3 ',
 '\n   ',
 'Name: My image 4 ',
 '\n   ',
 'Name: My image 5 ',
 '\n  ']

<b>foo::text</b> trả về kết quả không nếu phần tử foo tồn tại nhưng không chứa text.
<br/>
Sử dụng <b>default=''</b> nếu bạn muốn trả về string.

In [19]:
sel.css('img::text').getall()

[]

In [20]:
sel.css('img::text').get(default='')

''

<b>a::attr(href)</b> chọn giá trị thuộc tính href của links:

In [21]:
sel.css('a::attr(href)').getall()

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']